# Read and write to Google Sheets with Python

## Setup

Install the necessary Google Python libraries:

```sh
pip install — upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
```

In [1]:
# Import standard libraries
import pandas as pd
import numpy as np
import pickle
import os

# Import Google API libraries
from googleapiclient import discovery
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

## Connection

Enable the Google Sheets API for your account and download credentials as a JSON file, by following [these steps](https://developers.google.com/sheets/api/quickstart/python#step_1_turn_on_the).

In [2]:
# Function to generate user token
def gsheet_api_check(SCOPES, TOKEN, CREDENTIALS):
    creds = None
    if os.path.exists(TOKEN):
        with open(TOKEN, 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CREDENTIALS, SCOPES)
            creds = flow.run_console()
        with open(TOKEN, 'wb') as token:
            pickle.dump(creds, token)
    return creds

# Connect to Google Sheets API v4
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']   # Allow read and write to Google Sheets
TOKEN = 'data/gsheet_token.pickle'
CREDENTIALS = 'data/gsheet_credentials.json'
CREDS = gsheet_api_check(SCOPES, TOKEN, CREDENTIALS)

In [3]:
# Connect to the service
service = discovery.build('sheets', 'v4', credentials=CREDS)
SPREADSHEET_ID = '1c0HS6Wc09bri6vvQQds3zVMGvXNbYJ3gKY5wtyaAYxI'   # Found in the Google Sheet URL
RANGE = 'Sheet1!A:Z'   # Tab and cells range to be read and/or written

## Read from a Google Sheet

In [4]:
# Read from a Google Sheet
request = service.spreadsheets().values().get(
    spreadsheetId=SPREADSHEET_ID, 
    range=RANGE
)
response = request.execute()

# Convert data to a pandas DataFrame
pd.DataFrame(response.get('values', []))

,0,1,2,3
0,A,B,C,D
1,"0,206","0,661",None,None
2,"0,144","0,157",None,None
3,"0,733","0,094",None,None
4,"0,18","0,607",None,None
5,"0,321","0,541",None,None


## Write a pandas DataFrame to a Google Sheet

In [5]:
# Optional: clear data on a Sheet range
request = service.spreadsheets().values().clear(
    spreadsheetId=SPREADSHEET_ID, 
    range=RANGE
)
response = request.execute()

In [7]:
# Create DataFrame to be written to the Google Sheet
df = pd.DataFrame(np.random.random(size=(5, 4)), columns=list('ABCD')).round(3)

# Convert DataFrame to a list of rows, with the columns names as first row
value_range_body = {
  "values": [list(df.columns)] + df.values.tolist()
}

# Write data to the Google Sheet
request = service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID, 
    range=RANGE, 
    valueInputOption='RAW', 
    body=value_range_body)
response = request.execute()